In [1]:
import sys 
sys.path.append('../modelling/')

In [2]:
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [3]:
model = LLM_MultitaskMultimodal.load_from_checkpoint('../modelling/models/multitask_multimodal_multilingual/version_9/epoch=1-step=2600.ckpt/pytorch_model.bin')

INFO:root:Unused kwargs when getting google/mt5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_singlemodal_wishquery2tax': None}, 'head_dict': {}}


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
model.transformer.save_pretrained('hf_models/query_classify_v2')

In [5]:
model.tokenizer.save_pretrained('hf_models/query_classify_v2')

('hf_models/query_classify_v2/tokenizer_config.json',
 'hf_models/query_classify_v2/special_tokens_map.json',
 'hf_models/query_classify_v2/spiece.model',
 'hf_models/query_classify_v2/added_tokens.json',
 'hf_models/query_classify_v2/tokenizer.json')

In [6]:
del model 
import gc 
gc.collect()

518

In [7]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hf_models/query_classify_v2")
model_ort = ORTModelForSeq2SeqLM.from_pretrained("hf_models/query_classify_v2", from_transformers=True,
    provider='CUDAExecutionProvider')


/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/modeling_utils.py:701: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:
In-place op on output of tensor.shape. See https://pytorch.org/docs/master/onnx.html#avoid-inplace-operations-when-using-tensor-shape-in-tracing-mode
In-place op on output of tensor.shape. See https://pytorch.org/docs/master/onnx.html#avoid-inplace-operations-when-using-tensor-shape-in-tracing-mode
2023-01-26 02:40:17.987894593 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.

In [8]:
model_ort.save_pretrained("hf_models/query_classify_v2")

In [9]:
from optimum.onnxruntime import ORTOptimizer, OptimizationConfig

In [10]:
optimizer = ORTOptimizer.from_pretrained(model_ort)

In [14]:
optimization_config = OptimizationConfig(
    optimization_level=99,
    enable_transformers_specific_optimizations=True,
    optimize_for_gpu=True
)

In [15]:
save_dir = 'hf_models/query_classify_v2_optimized'

In [16]:
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

2023-01-26 02:45:56.273858090 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-01-26 02:45:56.275710984 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2023-01-26 02:45:57.140667974 [W:onnxruntime:, inference_session.cc:1458 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.
INFO:fusion_utils:Remove reshape node /Reshape since its input shape is same as output: ['batch_size', 'encoder_sequence_length']
INFO:onnx_model:Graph pruned: 0 inputs,

PosixPath('hf_models/query_classify_v2_optimized')